In [1]:
from bs4 import BeautifulSoup
from os.path import expanduser
import os
from tqdm import tqdm
import pandas as pd
import requests
import urllib3
import time
import lxml

In [2]:
BASEURL = "https://www.cia.gov"

SCIAB = ['scientific-abstracts', 1653]
GREC = ['general-cia-records', 38439]
JPRS = ['jprs', 127]
LOC = ['library-congress', 394]
NIS = ['nis', 22]

FIELDS = ['id','title','classification','publication_date','page_count','content_type']

In [3]:
def gather_meta(pager):
    df = pd.DataFrame(columns=FIELDS)
    idx = 0
    
    for i in tqdm(range(pager[1])):
        
        pagination_link = BASEURL + '/library/readingroom/collection/' + pager[0] + '?page=' + str(i)
        pagination_page = requests.get(pagination_link)
        p_soup = BeautifulSoup(pagination_page.content, 'lxml') # pagination soup
        
        for doc in p_soup.find_all("h4", class_="field-content"):
            
            PUB_DATE, ID, CLASSIFICATION, PAGE_COUNT, CONTENT_TYPE = ""
            
            a = doc.select_one("a")
            link = str(a.get('href'))
            TITLE = str(a.string or "")
            
            doc_page = requests.get(BASEURL + link)
            m_soup = BeautifulSoup(doc_page.content, 'lxml') # metadata soup
            time.sleep (.05)
        
            try:
                PUB_DATE = str(m_soup.select_one(".field-name-field-pub-date").select_one("span").get('content'))
                ID = str(m_soup.select_one(".field-name-field-document-number").select_one(".field-item").string)
                CLASSIFICATION = str(m_soup.select_one(".field-name-field-original-classification").select_one(".field-item").string)
                PAGE_COUNT = str(m_soup.select_one(".field-name-field-page-count").select_one(".field-item").string)
                CONTENT_TYPE = str(m_soup.select_one(".field-name-field-content-type").select_one(".field-item").string)
            
            df.loc[idx] = [ID, TITLE, CLASSIFICATION, PUB_DATE, PAGE_COUNT, CONTENT_TYPE]
            idx+=1  
            
    return df

In [4]:
def totalpages(df):
    return df.page_count.astype(int).sum()

In [5]:
def values(df, col):
    return str(df[col].unique())

In [6]:
jprs = gather_meta(JPRS)
# jprs.to_csv("jprs.csv")
jprs.head()

100%|██████████| 127/127 [39:26<00:00, 18.63s/it]


,id,title,classification,publication_date,page_count,content_type
0,CIA-RDP82-00850R000400050051-6,JPRS ID: 10001 USSR REPORT ECONOMIC AFFAIRS D...,U,,31,REPORTS
1,CIA-RDP82-00850R000400050053-4,JPRS ID: 10003 SOUTH AND EAST ASIA REPORT,U,,17,REPORTS
2,CIA-RDP82-00850R000400050055-2,"JPRS ID: 10005 CHINA REPORT POLITICAL, SOCIOL...",U,,42,REPORTS
3,CIA-RDP82-00850R000400050057-0,JPRS ID: 10007 WEST EUROPE REPORT,U,,33,REPORTS
4,CIA-RDP82-00850R000400050058-9,JPRS ID: 10008 SUB-SAHARAN AFRICA REPORT,U,,72,REPORTS


In [ ]:
nis = gather_meta(NIS)
# nis.to_csv("nis.csv")
nis.head()

In [ ]:
sciab = gather_meta(SCIAB)
# sciab.to_csv("sciab.csv")
sciab.head()

In [ ]:
grec = gather_meta(GREC)
# grec.to_csv("grec.csv")
grec.head()